## Read Transformed Descriptions

# see these articles
* https://medium.com/@ppasumarthi_69210/word-embeddings-in-keras-be6bb3092831   <== This is the good one

* https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/
* https://stackoverflow.com/questions/52126539/using-pretrained-gensim-word2vec-embedding-in-keras

In [1]:
from os import listdir
from os.path import isfile, join

In [2]:
path = './../data/word_encoding/'

In [3]:
files = [f for f in listdir(path) if isfile(join(path, f))]

In [4]:
import pandas as pd
import numpy as np

In [5]:
df_temp = pd.read_pickle(path+files[0])

In [6]:
df_temp.word_vec.values[1].shape

(3650, 300)

In [7]:
X = df_temp.word_vec.values

##### Padding to max length

In [8]:
max_length = np.max([m.shape[0] for m in X])

In [9]:
X_new = [np.concatenate((x,np.zeros((max_length-x.shape[0],300)))) for x in X] 

In [10]:
X_new[0].shape

(5593, 300)

In [16]:
X_new1 = np.asarray(X_new)

In [27]:
X_new1.shape

(50, 5593, 300)

In [30]:
X_new2 = np.reshape(X_new1,(50,-1))

In [31]:
X_new2.shape

(50, 1677900)

In [32]:
X_new2[0]

array([-0.14160156, -0.07226562,  0.11572266, ...,  0.        ,
        0.        ,  0.        ])

###### Model - Simple DNN

In [20]:
from keras.layers import Input, Dense,Conv2D,MaxPooling2D,UpSampling2D
from keras.models import Model

input_ = Input(shape=X_new1.shape)
x = Conv2D(16,(1,1), activation='relu', padding='same')(input_)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8,(3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same', name='encoder')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_new1,X_new1)

ValueError: Error when checking input: expected input_5 to have 4 dimensions, but got array with shape (50, 5593, 300)

In [21]:
autoencoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 50, 5593, 300)     0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 50, 5593, 16)      4816      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 25, 2797, 16)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 25, 2797, 8)       1160      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 1399, 8)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 13, 1399, 8)       584       
_________________________________________________________________
encoder (MaxPooling2D)       (None, 7, 700, 8)         0   

In [81]:
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('encoder').output)

In [82]:
encoder

In [84]:
X_new1[0].shape

(5593, 300)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 50 arrays: [array([[-0.14160156, -0.07226562,  0.11572266, ...,  0.06884766,
         0.07666016,  0.08105469],
       [-0.1640625 ,  0.08203125,  0.02819824, ...,  0.03515625,
        -0.13769531,  0.16796875],...